In [2]:
from keras.callbacks import ModelCheckpoint
from keras.models import Model
from keras.layers import Input, concatenate, Dropout, Reshape, Permute, Activation, ZeroPadding2D, Cropping2D
from keras.layers.convolutional import Conv2D, MaxPooling2D, UpSampling2D, Conv2DTranspose
from keras.regularizers import l2
from keras.preprocessing.image import ImageDataGenerator
import numpy as np

In [3]:
IMG_HEIGHT = 256
IMG_WIDTH = 256
BATCH_SIZE = 16

x_train = np.load('data/x_train.npz')['data'].astype(np.float32)
y_train = np.load('data/y_train.npz')['data'].astype(np.float32)
x_val = np.load('data/x_val.npz')['data'].astype(np.float32)
y_val = np.load('data/y_val.npz')['data'].astype(np.float32)

print(x_train.shape, y_train.shape)
print(x_val.shape, y_val.shape)

(12706, 256, 256, 3) (12706, 256, 256, 2)
(5000, 256, 256, 3) (5000, 256, 256, 2)


In [4]:
train_datagen = ImageDataGenerator(
    rescale=1./255,
    brightness_range=[0.8, 1.2]
)

val_datagen = ImageDataGenerator(
    rescale=1./255
)

train_gen = train_datagen.flow(
    x_train,
    y_train,
    batch_size=BATCH_SIZE,
    shuffle=True
)

val_gen = val_datagen.flow(
    x_val,
    y_val,
    batch_size=BATCH_SIZE,
    shuffle=False
)

In [5]:
inputs = Input(shape=(IMG_HEIGHT, IMG_WIDTH, 3))

# encode
# 256x256
conv1 = Conv2D(32, (3, 3), activation='relu', padding='same', kernel_initializer='orthogonal')(inputs)
conv1 = Dropout(0.2)(conv1)
conv1 = Conv2D(64, (3, 3), activation='relu', padding='same', kernel_initializer='orthogonal')(conv1)
pool1 = MaxPooling2D(pool_size=(2, 2))(conv1)

# 128x128
conv2 = Conv2D(64, (3, 3), activation='relu', padding='same', kernel_initializer='orthogonal')(pool1)
conv2 = Dropout(0.2)(conv2)
conv2 = Conv2D(128, (3, 3), activation='relu', padding='same', kernel_initializer='orthogonal')(conv2)
pool2 = MaxPooling2D(pool_size=(2, 2))(conv2)

# 64x64
conv3 = Conv2D(128, (3, 3), activation='relu', padding='same', kernel_initializer='orthogonal')(pool2)
conv3 = Dropout(0.2)(conv3)
conv3 = Conv2D(256, (3, 3), activation='relu', padding='same', kernel_initializer='orthogonal')(conv3)
pool3 = MaxPooling2D(pool_size=(2, 2))(conv3)

# decode
up8 = UpSampling2D(size=(2, 2))(pool3)
up8 = concatenate([up8, conv3], axis=-1)
conv8 = Conv2D(128, (3, 3), activation='relu', padding='same', kernel_initializer='orthogonal')(up8)
conv8 = Dropout(0.2)(conv8)
conv8 = Conv2D(128, (3, 3), activation='relu', padding='same', kernel_initializer='orthogonal')(conv8)

up9 = UpSampling2D(size=(2, 2))(conv8)
up9 = concatenate([up9, conv2], axis=-1)
conv9 = Conv2D(64, (3, 3), activation='relu', padding='same', kernel_initializer='orthogonal')(up9)
conv9 = Dropout(0.2)(conv9)
conv9 = Conv2D(64, (3, 3), activation='relu', padding='same', kernel_initializer='orthogonal')(conv9)

up10 = UpSampling2D(size=(2, 2))(conv9)
up10 = concatenate([up10, conv1], axis=-1)
conv10 = Conv2D(32, (3, 3), activation='relu', padding='same', kernel_initializer='orthogonal')(up10)
conv10 = Dropout(0.2)(conv10)
conv10 = Conv2D(32, (3, 3), activation='relu', padding='same', kernel_initializer='orthogonal')(conv10)

conv11 = Conv2D(2, (1, 1), padding='same', activation='relu',
                kernel_initializer='he_normal', kernel_regularizer=l2(0.005))(conv10)
conv11 = Reshape((IMG_HEIGHT * IMG_WIDTH, 2))(conv11)
# conv11 = Reshape((2, IMG_HEIGHT * IMG_WIDTH))(conv11)
# conv11 = Permute((2, 1))(conv11)

conv11 = Activation('softmax')(conv11)

outputs = Reshape((IMG_HEIGHT, IMG_WIDTH, 2))(conv11)

model = Model(inputs=inputs, outputs=outputs)

model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
model.summary()

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            (None, 256, 256, 3)  0                                            
__________________________________________________________________________________________________
conv2d_1 (Conv2D)               (None, 256, 256, 32) 896         input_1[0][0]                    
__________________________________________________________________________________________________
dropout_1 (Dropout)             (None, 256, 256, 32) 0           conv2d_1[0][0]                   
__________________________________________________________________________________________________
conv2d_2 (Conv2D)               (None, 256, 256, 64) 18496       dropout_1[0][0]                  
__________________________________________________________________________________________________
max_poolin

In [6]:
history = model.fit_generator(train_gen,
    validation_data=val_gen,
    epochs=20,
    callbacks=[
        ModelCheckpoint('models/unet.h5', monitor='val_acc', save_best_only=True, verbose=1)
    ]
)

Epoch 1/20
795/795 [==============================] - 222s 279ms/step - loss: 0.2172 - acc: 0.8989 - val_loss: 0.1399 - val_acc: 0.9426

Epoch 00001: val_acc improved from -inf to 0.94261, saving model to models/unet.h5
Epoch 2/20
795/795 [==============================] - 221s 277ms/step - loss: 0.1367 - acc: 0.9438 - val_loss: 0.1055 - val_acc: 0.9592

Epoch 00002: val_acc improved from 0.94261 to 0.95920, saving model to models/unet.h5
Epoch 3/20
795/795 [==============================] - 223s 280ms/step - loss: 0.1033 - acc: 0.9595 - val_loss: 0.0825 - val_acc: 0.9684

Epoch 00003: val_acc improved from 0.95920 to 0.96843, saving model to models/unet.h5
Epoch 4/20
795/795 [==============================] - 224s 281ms/step - loss: 0.0879 - acc: 0.9661 - val_loss: 0.0732 - val_acc: 0.9727

Epoch 00004: val_acc improved from 0.96843 to 0.97265, saving model to models/unet.h5
Epoch 5/20
795/795 [==============================] - 222s 279ms/step - loss: 0.0788 - acc: 0.9699 - val_loss: 